In [20]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import seaborn as sln
import matplotlib.pyplot as plt
%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['sample_submission.csv', 'test.csv', 'train.csv']


In [30]:
train=pd.read_csv('../input/train.csv')

In [ ]:
test=pd.read_csv('../input/test.csv')

In [ ]:
train.info()

In [31]:
train.head ()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:41.987077+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:42.870083+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:45.222647+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:47.601894+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-29 10:50:48.488476+00,2,NaN,2006,rejected,0,0,0,1,0,0.0,4,47


In [ ]:
train.isnull().sum()

Lets build a decision tree

In [ ]:
diff_bias=[]
for i in  train.columns:
    if train[i].isnull().sum()==1399744:
        diff_bias.append(i)
        

In [ ]:
tree_df=pd.concat([train[diff_bias],train['target']],axis=1)

In [ ]:
tree_df.head()

In [ ]:
tree_df.shape

In [ ]:
tree_df=tree_df.dropna()

In [ ]:
tree_df.shape

In [ ]:
tree_df['target'].value_counts()

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree

In [ ]:
#for visulaization keep depth low
dt=DecisionTreeRegressor(max_depth=4,min_samples_split=10,max_features=10)

In [ ]:
clf=dt.fit(tree_df.drop('target',axis=1),tree_df['target'])

In [ ]:
import graphviz 
dot_data = tree.export_graphviz(clf, out_file='tree.dot') 

In [ ]:
from sklearn.tree import export_graphviz
from IPython.display import SVG

In [ ]:
from graphviz import Source
from IPython.display import display

In [ ]:
graph=Source(tree.export_graphviz(clf,feature_names=tree_df.drop('target',axis=1).columns))

In [ ]:
display(SVG(graph.pipe(format='svg')))

## Exploring Text Data

In [ ]:
train['comment_text'][0]

In [29]:
train.head(5)

AttributeError: module 'fastai.train' has no attribute 'head'

In [ ]:
full_text=' '
for i in train[train['article_id']==2006]['comment_text']:
    full_text+=i
    

In [ ]:
train['comment_text'][4]

In [ ]:
train['comment_text'][5]

In [ ]:
train['comment_text'][6]

In [ ]:
#lets see the effect of gender on toxic comments
train_new=train.dropna()


In [ ]:
sln.jointplot(train['target'],train['severe_toxicity'])

In [ ]:
sln.distplot(train_new['female'])

Using ULMFit

In [ ]:
#Before checking the cuttoff lets see
#lets clean the data first

In [22]:
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer

In [32]:
stop = stopwords.words('english')
train['comm_clean'] = train['comment_text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [ ]:
sln.distplot(train['target'])

In [33]:
# cuttoff-0.5
train['target_cat']=np.where(train['target']>0.5,1,0)

In [25]:
# import libraries
import fastai
from fastai import *
from fastai.text import * 
from functools import partial


In [38]:
df.to_feather('cleaned_df')

In [36]:
df=train[['comm_clean','target_cat']]

In [37]:
from sklearn.model_selection import train_test_split

# split data into training and validation set
df_trn, df_val = train_test_split(df, stratify = df['target_cat'], test_size = 0.4, random_state = 12)